In [2]:
import numpy as np
import icecube
from icecube import dataio, dataclasses, icetray

def files(args, include_frames=[], print_file=False):
    """A frame generator that can continue over multiple files"""
    if not isinstance(args, list):
        args = [args]
    
    for a in args:
        try:
            with dataio.I3File(a) as i3file:
                if print_file: print(f"Opening: {a}")
                for frame in i3file:
                    if len(include_frames) and not frame.Stop.id in include_frames:
                        continue
                    yield frame
        except RuntimeError:
            print(f"Error opening file: {a}")
            pass

def process_frame(frame):
    """Extract relevant particles (neutrinos + muons) from an air shower."""
    if "I3MCTree" not in frame:
        return None
    
    mctree = frame["I3MCTree"]
    new_mctree = dataclasses.I3MCTree()
    
    for particle in mctree:
        if abs(particle.pdg_encoding) in [12, 14, 16, 13]:  # Neutrinos and muons
            new_mctree.insert(particle)
            
    new_frame = icetray.I3Frame(icetray.I3Frame.DAQ)
    new_frame["I3MCTree"] = new_mctree
    return new_frame

def process_files(input_files, output_file):
    """Process CORSIKA MC files and store selected particles in an output I3 file."""
    outfile = dataio.I3File(output_file, "w")
    
    for frame in files(input_files, print_file=True):
        new_frame = process_frame(frame)
        if new_frame:
            outfile.push(new_frame)
    
    outfile.close()
    print(f"Processed frames saved to {output_file}")



In [3]:
input_file='/data/sim/IceCube/2023/generated/CORSIKA-in-ice/22803/0000000-0000999/IC86.2023_corsika.022803.000000.i3.zst'
output_file='output_test.i3.zst'
process_files(input_file, output_file)

Opening: /data/sim/IceCube/2023/generated/CORSIKA-in-ice/22803/0000000-0000999/IC86.2023_corsika.022803.000000.i3.zst
Processed frames saved to output_test.i3.zst


In [4]:
file = dataio.I3File('output_test.i3.zst')
n=0
while file.more():
    frame=file.pop_frame()
    if 'I3MCTree' in frame:
        n=n+1
n

23995

In [6]:
frame.keys()

['I3MCTree']

In [7]:
print(frame['I3MCTree'])

[I3MCTree:
  25713424 NuMu (1740.15m, -7.04928m, 1949.84m) (40.2487deg, 352.083deg) 3520.21ns 463.038GeV nanm
  25713973 NuMuBar (505.02m, 177.41m, 467.177m) (79.8808deg, 162.277deg) 11183.9ns 0.0157128GeV 0m
  25713972 NuE (505.02m, 177.41m, 467.177m) (106.006deg, 26.4295deg) 11183.9ns 0.0499454GeV 0m
  25713970 MuPlus (542.966m, 172.061m, 512.5m) (40.2147deg, 351.977deg) 9793.99ns 12.7871GeV 59.3526m
  25713968 MuPlus (607.695m, 162.938m, 589.814m) (40.2147deg, 351.977deg) 9456.27ns 36.7296GeV 101.245m
  25713966 MuPlus (623.373m, 160.728m, 608.539m) (40.2147deg, 351.977deg) 9374.47ns 43.9149GeV 24.5212m
  25713964 MuPlus (658.395m, 155.792m, 650.371m) (40.2147deg, 351.977deg) 9191.74ns 56.8061GeV 54.7799m
  25713962 MuPlus (733.413m, 145.218m, 739.974m) (40.2147deg, 351.977deg) 8800.35ns 83.6287GeV 117.338m
  25713960 MuPlus (752.52m, 142.525m, 762.795m) (40.2147deg, 351.977deg) 8700.66ns 91.0664GeV 29.8847m
  25713958 MuPlus (783.669m, 138.134m, 800m) (40.2147deg, 351.977deg) 8538.